# Groundedness Checking Skills

In this notebook we run a simple grounding pipeline, to see if a summary text has any ungrounded additions as compared to 

Let us first define our grounding text:

In [1]:
grounding_text = """I am by birth a Genevese, and my family is one of the most distinguished of that republic.
My ancestors had been for many years counsellors and syndics, and my father had filled several public situations
with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention
to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety
of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband
and the father of a family.

As the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his
most intimate friends was a merchant who, from a flourishing state, fell, through numerous mischances, into poverty.
This man, whose name was Beaufort, was of a proud and unbending disposition and could not bear to live in poverty
and oblivion in the same country where he had formerly been distinguished for his rank and magnificence. Having
paid his debts, therefore, in the most honourable manner, he retreated with his daughter to the town of Lucerne,
where he lived unknown and in wretchedness. My father loved Beaufort with the truest friendship and was deeply
grieved by his retreat in these unfortunate circumstances. He bitterly deplored the false pride which led his friend
to a conduct so little worthy of the affection that united them. He lost no time in endeavouring to seek him out,
with the hope of persuading him to begin the world again through his credit and assistance.

Beaufort had taken effectual measures to conceal himself, and it was ten months before my father discovered his
abode. Overjoyed at this discovery, he hastened to the house, which was situated in a mean street near the Reuss.
But when he entered, misery and despair alone welcomed him. Beaufort had saved but a very small sum of money from
the wreck of his fortunes, but it was sufficient to provide him with sustenance for some months, and in the meantime
he hoped to procure some respectable employment in a merchant's house. The interval was, consequently, spent in
inaction; his grief only became more deep and rankling when he had leisure for reflection, and at length it took
so fast hold of his mind that at the end of three months he lay on a bed of sickness, incapable of any exertion.

His daughter attended him with the greatest tenderness, but she saw with despair that their little fund was
rapidly decreasing and that there was no other prospect of support. But Caroline Beaufort possessed a mind of an
uncommon mould, and her courage rose to support her in her adversity. She procured plain work; she plaited straw
and by various means contrived to earn a pittance scarcely sufficient to support life.

Several months passed in this manner. Her father grew worse; her time was more entirely occupied in attending him;
her means of subsistence decreased; and in the tenth month her father died in her arms, leaving her an orphan and
a beggar. This last blow overcame her, and she knelt by Beaufort's coffin weeping bitterly, when my father entered
the chamber. He came like a protecting spirit to the poor girl, who committed herself to his care; and after the
interment of his friend he conducted her to Geneva and placed her under the protection of a relation. Two years
after this event Caroline became his wife."""

And the summary we're going to examine:

In [2]:
summary_text = """The narrator's father was a respected public figure in Geneva. He was friends with a wealthy
merchant named Beaufort, who fell into poverty and retreated to Lucerne with his daughter. The narrator's father
searched for ten months before finding Beaufort, who was sick and unable to work. Beaufort died after several
months, leaving his daughter an orphan and a beggar. The narrator's father took her under his care and eventually
married her two years later."""

## Set up Semantic Kernel

We prepare our kernel in the usual way:

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

## Import the Skill

We have to import the semantic and native skills in separate calls:

In [4]:
from semantic_kernel.core_skills.text_skill import TextSkill

# note: using skills from the samples folder
skills_directory = "../../skills"

groundingSemanticFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "GroundingSkill")
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
#writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
#text_skill = kernel.import_skill(TextSkill(), "TextSkill")

In [5]:
entity_extraction = groundingSemanticFunctions["ExtractEntities"]
reference_check = groundingSemanticFunctions["ReferenceCheckEntities"]
entity_excision = groundingSemanticFunctions["ExciseEntities"]

## Preparing the context

We now create the context in which will will operate. In the beginning, this just specifies the kind of entities we want to check for grounding:

In [6]:
context = kernel.create_new_context()
context["topic"] = "people and places"
context["example_entities"] = "names of people, familial relationships and towns"

## Extracting the entities

We're now ready to get the entities from the summary text:

In [7]:
extraction_result = entity_extraction(summary_text, context=context)

Examine the raw result

In [8]:
print(extraction_result.result)

- Narrator's father
- Geneva
- Beaufort
- Lucerne
- Daughter of Beaufort
- Narrator's father's care


## Reference cross check

We now want to see if these entities appear in our grounding text. First, we place the grounding text into the general milieu of our kernel, so it can form background information:

In [9]:
context["reference_context"] = grounding_text

With our grounding information in place, we can run the reference checking function (you didn't think that the pretty printing of the list of extracted entities was for _your_ benefit, did you?):

In [10]:
grounding_result = reference_check(extraction_result.result, context=context)

print(grounding_result.result)


- Narrator's father's care


## Excision

We can also try removing the extra entities:

In [11]:
context["target_entities"] = grounding_result.result

excision_result = entity_excision(summary_text, context=context)

print(grounding_result.result)


The narrator's father was a respected public figure in Geneva. He was friends with a wealthy merchant named Beaufort, who fell into poverty and retreated to Lucerne with his daughter. The narrator's father searched for ten months before finding Beaufort, who was sick and unable to work. Beaufort died after several months, leaving his daughter an orphan and a beggar. The narrator's father took her in and eventually married her two years later.


## Using a Planner

In [12]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [13]:
summary_text = "I like cars and trucks"
grounding_text = """My sister and I both like aeroplanes and trucks. I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation."""

target_topic = "transportation"

In [14]:
ask = f"""Make a list of things related to {target_topic} which are in the summary
but which are not grounded in the original. Do not do anything else after extracting
the list of things related to {target_topic}.

This is the summary text:

[SUMMARY_TEXT]
{summary_text}
[/SUMMARY_TEXT]

This was based on the following original text:

[ORIGINAL_TEXT]
{grounding_text}
[/ORIGINAL_TEXT]
"""

print(ask)

Make a list of things related to transportation which are in the summary
but which are not grounded in the original. Do not do anything else after extracting
the list of things related to transportation.

This is the summary text:

[SUMMARY_TEXT]
I like cars and trucks
[/SUMMARY_TEXT]

This was based on the following original text:

[ORIGINAL_TEXT]
My sister and I both like aeroplanes and trucks. I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation.
[/ORIGINAL_TEXT]



Write a new prompt for the planner, with the goal of getting better plans:

In [15]:
PROMPT = """
# Task

1. You are a planner for the Semantic Kernel.
2. Your job is to create a properly formatted JSON plan step by step, to satisfy the goal given.
3. Create a list of subtasks based off the [GOAL] provided.
4. Each subtask must be from within the [AVAILABLE_FUNCTIONS] list. Do not use any functions that are not in the list.
5. Base your decisions on which functions to use from the description and the name of the function.
6. Sometimes, a function may take arguments. Provide them if necessary.
7. The plan should be as short as possible.
8. The plan does not have to use all of the [AVAILABLE_FUNCTIONS]
9. Ensure that the plan you project is valid JSON. Remove new line characters if necessary

Here are some examples:

## Example 1

[AVAILABLE_FUNCTIONS]
EmailConnector.LookupContactEmail
description: looks up the a contact and retrieves their email address
args:
- name: the name to look up

WriterSkill.EmailTo
description: email the input text to a recipient
args:
- input: the text to email
- recipient: the recipient's email address. Multiple addresses may be included if separated by ';'.

WriterSkill.Translate
description: translate the input to another language
args:
- input: the text to translate
- language: the language to translate to

WriterSkill.Summarize
description: summarize input text
args:
- input: the text to summarize

FunSkill.Joke
description: Generate a funny joke
args:
- input: the input to generate a joke about
[/AVAILABLE_FUNCTIONS]

[GOAL]
Tell a joke about cars. Translate it to Spanish
[/GOAL]

[OUTPUT]
    {
        "input": "cars",
        "subtasks": [
            {"function": "FunSkill.Joke"},
            {"function": "WriterSkill.Translate", "args": {"language": "Spanish"}}
        ]
    }

## Example 2

[AVAILABLE_FUNCTIONS]
WriterSkill.Brainstorm
description: Brainstorm ideas
args:
- input: the input to brainstorm about

WriterSkill.FollowStyle
description: Rewrite a text in a given style
args:
- input: the original text
- text_sample: A sample of text whose style should be used to rewrite the input

WriterSkill.EmailTo
description: Write an email to a recipient
args:
- input: the input to write about
- recipient: the recipient's email address.

WriterSkill.Translate
description: translate the input to another language
args:
- input: the text to translate
- language: the language to translate to
[/AVAILABLE_FUNCTIONS]

[GOAL]
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Edgar Allen Poe so write using his style.
E-mail these ideas to my significant other. Translate it to French.

The following is a sample text by Edgar Allen Poe:

[SAMPLE_TEXT]
Take thy beak from out my heart, and take thy form from off my door!
Quoth the Raven, 'Nevermore.'
[/SAMPLE_TEXT]
[/GOAL]

[OUTPUT]
    {
        "input": "Valentine's Day Date Ideas",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "WriterSkill.FollowStyle", "args" : { "text_sample": "Take thy beak from out my heart, and take thy form from off my door!\nQuoth the Raven, 'Nevermore.'"}},
            {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
            {"function": "WriterSkill.Translate", "args": {"language": "French"}}
        ]
    }
    
## Example 3
    
[AVAILABLE_FUNCTIONS]
TextSkill.Append
description: append one text after another
args:
- input: the first text
- second_text: the text to place after the first

WriterSkill.Translate
description: translate the input to another language
args:
- input: the text to translate
- language: the language to translate to
[/AVAILABLE_FUNCTIONS]

[GOAL]
I have two texts from the same original piece. The
first needs to be translated into English, and then concatenated
with the second.

This is the first text:
[TEXT_A]
GALLIA est omnis divisa in partes tres, quarum unam incolunt Belgae,
aliam Aquitani, tertiam qui ipsorum lingua Celtae, nostra Galli appellantur
[/TEXT_A]

This is the second text:
[TEXT_B]
All these differ from each other in language, customs and laws
[/TEXT_B]
[/GOAL]

[OUTPUT]
    {
        "input": "GALLIA est omnis divisa in partes tres, quarum unam incolunt Belgae,\naliam Aquitani, tertiam qui ipsorum lingua Celtae, nostra Galli appellantur",
        "subtasks": [
            {"function": "WriterSkill.Translate", "args": { "language": "english" }},
            {"function": "TextSkill.Append", "args" : { "second_text": "All these differ from each other in language, customs and laws"}}
        ]
    }

# Task

Produce an execution plan for the following goal, based on these functions:

[AVAILABLE_FUNCTIONS]
{{$available_functions}}
[/AVAILABLE_FUNCTIONS]

[GOAL]
{{$goal}}
[/GOAL]

[OUTPUT]
"""

Now, generate the plan.
In theory, only two skills should be used; extract the entities, and then reference check them.
This might not :

In [16]:
my_plan = await planner.create_plan_async(ask, kernel, prompt=PROMPT)

print(my_plan.generated_plan)

{
    "input": "I like cars and trucks",
    "subtasks": [
        {
            "function": "GroundingSkill.ExtractEntities", 
            "args": {
                "topic": "Transportation", 
                "example_entities": ["aeroplanes", "trucks"]
            }
         },
        {
            "function": "GroundingSkill.ReferenceCheckEntities",
            "args": {
                "reference_context": "My sister and I both like aeroplanes and trucks. I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation."
            }
        }
    ]
}


Run the plan and see what we get:

In [17]:
results = await planner.execute_plan_async(my_plan, kernel, debug_print=True)

print(results)

GroundingSkill.ExtractEntities: BEGIN
Context input: I like cars and trucks
Context topic: Transportation
Context example_entities: ['aeroplanes', 'trucks']
GroundingSkill.ExtractEntities output: I like cars and trucks
GroundingSkill.ReferenceCheckEntities: BEGIN
Context input: I like cars and trucks
Context topic: Transportation
Context example_entities: ['aeroplanes', 'trucks']
Context reference_context: My sister and I both like aeroplanes and trucks. I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation.
GroundingSkill.ReferenceCheckEntities output: 
- I like cars

- I like cars


## A More Complicated Plan and new skills

Now, let's ask for something more complicated.
We can give the planner a longer text, ask it to summarise the text, then extract entities, cross check them, and remove any which shouldn't be there.
However, there's a problem with doing this: how to pass the summary around.

The planner we're using makes the output of each function into the input of the next. Other arguments can be passed in via the context, but these have to be known at the time the planner is invoked; they can't be previous outputs.
This gives us a problem for the task described above; the summary has to be passed into both the 'extract entities' stage, and the 'excise entities' stage, but is itself the output of the 'summarise text' stage.
For this, we're going to make use of variants of the previous skills, which will keep the summary in the output, and pass it through.

Let's start by describing what we want to do:

In [29]:
target_topic_2 = "people and places"

ask2 = f"""
Summarise the text between [TEXT] and [/TEXT]. Then make a list of entities related to {target_topic_2} in
the summary. Check this list against the text given below, and make a list of the entities which are in the
summary but not the original text. Take this list and rewrite the summary to remove those entities.

This is the text to summarise:

[TEXT]
{grounding_text}
[/TEXT]
"""

print(ask2)


Summarise the text between [TEXT] and [/TEXT]. Then make a list of entities related to people and places in
the summary. Check this list against the text given below, and make a list of the entities which are in the
summary but not the original text. Take this list and rewrite the summary to remove those entities.

This is the text to summarise:

[TEXT]
My sister and I both like aeroplanes and trucks. I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation.
[/TEXT]



Now, create a new semantic kernel, and add skills to it:

In [30]:
kernel_2 = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel_2.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel_2.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))
    
# note: using skills from the samples folder
skills_directory = "../../skills"

# Note change in directory
grounding2Skill = kernel_2.import_semantic_skill_from_directory(skills_directory, "Grounding2Skill")
summarize_skill = kernel_2.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")

### Separate Stages

As before, let's go through the pipeline we want stage by stage, before trying to use the planner.

First, a summary:

In [31]:
summary_result = summarize_skill['Summarize'](grounding_text).result

print(summary_result)


My sister and I both enjoy aeroplanes and trucks. I am from Geneva and my family is well-known and respected. My ancestors were counselors and syndics, and my father had held several public positions with distinction.


Now we extract entities:

In [32]:
context = kernel_2.create_new_context()
context["grounding_context"] = grounding_text
context["topic"] = "people and places"
context["example_entities"] = "names of people, familial relationships and towns"

entity_extraction_result = grounding2Skill['ExtractEntities'](summary_result, context=context).result

print(entity_extraction_result)

<entities>
- My sister
- Geneva
- My family
- My ancestors
- Counselors
- Syndics
- My father
</entities>

<context>
My sister and I both enjoy aeroplanes and trucks. I am from Geneva and my family is well-known and respected. My ancestors were counselors and syndics, and my father had held several public positions with distinction.
</context>


Note how we're repeating the summary, so we can pass it through. Now do the cross check:

In [33]:
reference_check_result = grounding2Skill['ReferenceCheckEntities'](entity_extraction_result, context=context).result

print(reference_check_result)


<ungrounded_entities>
- Counselors
- Syndics
</ungrounded_entities>

<context>
My sister and I both enjoy aeroplanes and trucks. I am from Geneva and my family is well-known and respected. My ancestors were counselors and syndics, and my father had held several public positions with distinction.
</context>


Finally try to excise the ungrounded entities:

In [34]:
excision_result = grounding2Skill['ExciseEntities'](reference_check_result, context=context).result

print(excision_result)


My sister and I both enjoy aeroplanes and trucks. I am from Geneva and my family is well-known and respected. My ancestors had held several public positions with distinction.


### Using the planner

Now let's go for broke:

In [37]:
my_plan_2 = await planner.create_plan_async(ask2, kernel_2, prompt=PROMPT)

print(my_plan_2.generated_plan)

{
    "input": "My sister and I both like aeroplanes and trucks. I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation.",
    "subtasks": [
        {"function": "SummarizeSkill.Summarize"},
        {"function": "Grounding2Skill.ExtractEntities", "args": {"topic": "people", "example_entities": ["Genevese", "family", "ancestors", "counsellors", "syndics", "father"]}},
        {"function": "Grounding2Skill.ReferenceCheckEntities", "args": {"reference_context": "My sister and I both like aeroplanes and trucks. I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation."}},
        {"function": "Grounding2Skill.ExciseEntities"}
    ]
}


In [38]:
results = await planner.execute_plan_async(my_plan_2, kernel_2, debug_print=True)

print(results)

SummarizeSkill.Summarize: BEGIN
Context input: My sister and I both like aeroplanes and trucks. I am by birth a Genevese, and my family is one of the most distinguished of that republic. My ancestors had been for many years counsellors and syndics, and my father had filled several public situations with honour and reputation.
SummarizeSkill.Summarize output: 
My sister and I both enjoy aeroplanes and trucks. My family is from Geneva and is well-known in the republic. My ancestors were counsellors and syndics, and my father had a successful career in public service.
Grounding2Skill.ExtractEntities: BEGIN
Context input: 
My sister and I both enjoy aeroplanes and trucks. My family is from Geneva and is well-known in the republic. My ancestors were counsellors and syndics, and my father had a successful career in public service.
Context topic: people
Context example_entities: ['Genevese', 'family', 'ancestors', 'counsellors', 'syndics', 'father']
Grounding2Skill.ExtractEntities output: 
My